## ¡Bienvenido a la aplicación de votación RASCAL-SAM!
##### El objetivo del proyecto es comparar el rendimiento a la hora de segmentar pulmones de un modelo de segmentación general como SAM con la segmentación llevada a cabo por un experto utilizando para ello las boundary box (recuadros rojos en las imágenes). Consiguiendo con ello un ahorro de tiempo mediante la segmentación por lotes.
### Indicaciones:

*   Dale al botón de 'Play' situado a la izquierda al lado de Mostrar código, aparecerá cuando sitúes el ratón encima <img src="https://icons.veryicon.com/png/o/miscellaneous/icon_1/play-118.png" width="20"/>
*   Inicia sesión con tu cuenta y acepta todos los permisos
*   ¡Comienza a votar!
*   No olvides clickar sobre "Actualizar calificación" ☝









In [ ]:
# @title
from datetime import datetime
from google.auth import default
from google.colab import auth
from google.colab import drive
drive.mount('/content/drive')
from googleapiclient.discovery import build
from io import BytesIO
from IPython.display import display, HTML
from IPython.display import display, Image
from ipywidgets import Button, HBox, VBox, Layout
from PIL import Image
from typing import Dict
import gspread
import gspread_dataframe as gd
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import os
import pandas as pd
import pytz
import random
import requests
import warnings
# TEST ROUTES
# ##Statics
folder_id = '1a2MHI1JHJkhmhJp__52ko6Xb9pkxuS05'
# Enlace a la carpeta compartida
folder_id_sam= '1VsGdmEUCfOhVMkrmIKm2zDjbglHCHwQD'
route_excel = 'https://docs.google.com/spreadsheets/d/1bkDSJvxhg-dXkN7qMuX3qZXQnjN9ZFJQtDuU4MDfgMw/edit?usp=sharing'
read_num_max_files_in_folder=1000  #max 1000 files
#FINAL ROUTES
##Statics
# folder_id = '1errSlmK5N3rCE34rXriFMV9CRHvCWX44'
# # Enlace a la carpeta compartida
# folder_id_sam= '1VsGdmEUCfOhVMkrmIKm2zDjbglHCHwQD'
# route_excel = 'https://docs.google.com/spreadsheets/d/1bkDSJvxhg-dXkN7qMuX3qZXQnjN9ZFJQtDuU4MDfgMw/edit?usp=sharing'

#uso horario
tz = pytz.timezone('Europe/Berlin')
warnings.filterwarnings("ignore")
###Authentication
# Autenticar la cuenta de Google Cloud
auth.authenticate_user()

# Verificar que la autenticación se realizó correctamente
print("Autenticación exitosa.")

# Comando para obtener el correo electrónico del usuario
user_email = os.popen("gcloud config get-value account").read()

# Imprimir el correo electrónico
print("Correo electrónico:", user_email)
##list all files in folder without limit
def get_all_files(service, folder_id):
    files = []
    page_token = None

    while True:
        try:
            response = service.files().list(
                q=f"'{folder_id}' in parents",
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=1000,
                fields="nextPageToken, files(id, name, webViewLink)",
                pageToken=page_token
            ).execute()

            files.extend(response.get('files', []))
            page_token = response.get('nextPageToken', None)

            if not page_token:
                break
        except HttpError as error:
            print(f'An error occurred: {error}')
            break
    return files
###Access image content
def access_to_images_content (folder_id_name: str) -> Dict [int, str]:
        # folder_id_name: ID de la carpeta compartida en Google Drive
        # Autenticación con Google Drive
        auth.authenticate_user()
        # Crear el servicio de Google Drive
        drive_service = build('drive', 'v3')
        # Listar el contenido de la carpeta
        list_of_dicts_files = get_all_files(drive_service, folder_id_name)
        transformed_dict = {index: item.get('name') for index, item in enumerate(list_of_dicts_files)}
        return transformed_dict

###images not yet voted by the authenticated user
def images_not_yet_voted (user_email: str,route_excel: str,nombres_con_clave: Dict) -> Dict[str, str]:
        creds, _ = default()
        gc = gspread.authorize(creds)
        wb = gc.open_by_url(route_excel) # A URL of your workbook.
        sheet1 = wb.worksheet('Hoja 1') # Enter your sheet name.
        # existing = gd.get_as_dataframe(sheet1,header=1,na_values=["#N/A"])
        existing = gd.get_as_dataframe(sheet1,header=0)
        existing.drop(existing.columns[existing.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
        existing = existing.dropna(how='all')
        images_voted_by_user_list= existing[existing['user']==user_email]
        images_voted_by_user_list = images_voted_by_user_list['image_name'].to_list()
        # print(images_voted_by_user_list)
        non_already_voted_images = {key: value for key, value in nombres_con_clave.items() if value not in images_voted_by_user_list}
        return non_already_voted_images
###get images
def get_image_from_shared_folder (shared_folder_id: str, image_name:str):
      # Autenticación
      auth.authenticate_user()
      # Crear el servicio de Google Drive
      drive_service = build('drive', 'v3')
      # Buscar la imagen por su nombre en la carpeta compartida
      results = drive_service.files().list(q=f"name='{image_name}' and '{shared_folder_id}' in parents",
                                          fields="nextPageToken,files(id, webContentLink)").execute()
      items = results.get('files', [])

      if items:
          # Obtener el enlace de descarga de la imagen
          image_link = items[0]['webContentLink']
          # Obtener la imagen desde el enlace compartido
          response = requests.get(image_link)
          image_data = response.content
          return image_data
          # Mostrar la imagen
          # display(Image(image_data, width=300, height=400))
      else:
          print("La imagen no fue encontrada en la carpeta compartida.")

###update results
def update_google_sheet (user_email: str, image_name: str, calificacion_sam: int, calificacion_experto: int, comment_1:str, comment_2:str, timestamp_now:str, route_excel: str) -> bool:
        creds, _ = default()

        gc = gspread.authorize(creds)
        wb = gc.open_by_url(route_excel) # A URL of your workbook.
        sheet1 = wb.worksheet('Hoja 1') # Enter your sheet name.
        # existing = gd.get_as_dataframe(sheet1,header=1,na_values=["#N/A"])
        existing = gd.get_as_dataframe(sheet1,header=0)
        existing.drop(existing.columns[existing.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
        existing = existing.dropna(how='all')
        existing_for_the_user = existing[existing['user']==user_email]
        if image_name not in existing_for_the_user['image_name'].values:
            # Leer datos del DataFrame
            data = {
                  'user': user_email,
                  'image_name': image_name,
                  'score_image_SAM': calificacion_sam,  # Puntuación inicial para score_image_SAM
                  'comments_sam': comment_1,  # Comentarios iniciales
                  'score_image_other': calificacion_experto,  # Puntuación inicial para score_image_other
                  'comments_expert': comment_2, # Comentarios iniciales,
                  'timestamp': timestamp_now
              }  # Reemplaza esto con tu DataFrame
            new_rows = pd.DataFrame(data, index=[0])

            # updated = existing.append(new_rows) ##pandas deprecated
            updated =pd.concat([existing, new_rows], ignore_index=True)


            gd.set_with_dataframe(sheet1, updated)
            return True
        else:
          return  False

###Votation app
nombres_imagenes = access_to_images_content(folder_id)
nombres_con_clave = {i: nombres_imagenes[i] for i in range(len(nombres_imagenes))}

# images without votation
nombres_con_clave_new = images_not_yet_voted(user_email,route_excel,nombres_con_clave)
# print(nombres_con_clave_new)

if not nombres_con_clave_new:
  display("Yas has votado todas las imágenes!")
else:
  claves = list(nombres_con_clave_new.keys())
  # print('total imagenes',nombres_imagenes)
  # getting random image without votes
  progress_bar = widgets.FloatProgress(
      value=len(claves),
      min=0,
      max=len(nombres_imagenes),
      step=1,
      bar_style='info',

   )
  progress_bar.description= "{:.2f}".format((progress_bar.value /progress_bar.max)*100) + '%'

  unicode_emojis_list = ["\U0001F973","\U0001F9D0","\U0001F917","\U0001F929","\U0001F609"]

  text_progress =  widgets.HTML(
    value=random.choice(unicode_emojis_list)+ "<b>Enhorabuena llevas </b>"+str(int(progress_bar.max)-int(progress_bar.value))+" de"+str(int(progress_bar.max))
    )
  display(text_progress)
  display(progress_bar)
  number_random = random.choice(claves)
  # ##routes
  image_data_expert = get_image_from_shared_folder (folder_id, nombres_con_clave_new[number_random])

  image_data_sam = get_image_from_shared_folder (folder_id_sam, nombres_con_clave_new[number_random])


  # print(folder_id_sam)
  # print(nombres_con_clave_new[number_random])
  dict_with_routes = {'expert':image_data_expert
                      ,'sam':image_data_sam}

  # print(dict_with_routes)
  ##mixed routes
  claves_mezcladas = list(dict_with_routes.keys())
  random.shuffle(claves_mezcladas)
  # Crear un nuevo diccionario con las claves mezcladas

  diccionario_mezclado = {clave: dict_with_routes[clave] for clave in claves_mezcladas}

  ##images
  list_of_images = [widgets.Image(
      value=list(diccionario_mezclado.values())[0],
      format='jpg',
      width=300,
      height=400,
  ),widgets.Image(
      value=list(diccionario_mezclado.values())[1],
      format='jpg',
      width=300,
      height=400,
  )]
  #sliders
  list_of_sliders = [widgets.IntSlider(min=1, max=8, step=1, description='Votación:'), widgets.IntSlider(min=1, max=8, step=1, description='Votación:')]
  l = Layout(flex='0 1 auto', height='200px', min_height='40px', width='45%')
  list_of_comments = [widgets.Textarea(value='Insert your comments for image', disabled=False,layout=l), widgets.Textarea(value='Insert your comments for image', disabled=False,layout=l)]

  #layouts
  first_layer_box = HBox([list_of_images[0], list_of_images[1]])
  second_layer_box = HBox([list_of_sliders[0], list_of_sliders[1]])
  third_layer_box = HBox([list_of_comments[0],list_of_comments[1]])
  all=VBox([first_layer_box, second_layer_box,third_layer_box])

  #style
  list_of_images[0].add_class("right-spacing-class")
  list_of_sliders[0].add_class("right-spacing-class")
  list_of_comments[0].add_class("right-spacing-class")

  pos_sam = list(diccionario_mezclado.keys()).index('sam')
  pos_expert = list(diccionario_mezclado.keys()).index('expert')


  display(HBox([all]))

  submit_button = widgets.Button(description='Actualizar Calificación',layout=Layout(margin='30px 0px 0px 0px', align_items='center' ))
  # submit_button.style.button_color = 'lightgreen'

  def on_submit_button_clicked(b):
       if update_google_sheet (user_email, nombres_con_clave_new[number_random], list_of_sliders[pos_sam].value, list_of_sliders[pos_expert].value, list_of_comments[pos_sam].value, list_of_comments[pos_expert].value, str(datetime.now(tz).strftime('%d-%m-%Y %H:%M:%S')), route_excel) == False:
            txt1 ='Imagen ya votada no se puede votar dos veces la misma imagen!!'
            display(txt1)

       else:
            display('Enhorabuena!Votación completada!!')


  submit_button.on_click(on_submit_button_clicked)


  display(submit_button)
  display(HTML(
      "<style>.right-spacing-class {margin-right: 70px; margin-bottom: 10px}</style>"
  ))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Autenticación exitosa.
Correo electrónico: paolavillarortega@gmail.com



HTML(value='🥳<b>Enhorabuena llevas </b>367 de1175')

FloatProgress(value=808.0, bar_style='info', description='68.77%', max=1175.0)

Button(description='Actualizar Calificación', layout=Layout(align_items='center', margin='30px 0px 0px 0px'), …